Il faut :
- prendre nos datas
- créer les emmbeddings avec la partie texte (extraire)
- faire une fonction pour trouver les similarités - faire ressortir les 3 documents les plus similaires par exemple
- fine tuner ensuite sur nos données- https://huggingface.co/blog/how-to-train-sentence-transformers
https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
https://www.codegram.com/blog/finding-similar-documents-with-transformers/
https://huggingface.co/facebook/bart-large-mnli
https://huggingface.co/docs/transformers/tasks/question_answering

% pip install -U sentence-transformers

In [6]:
from typing import List
from sentence_transformers import SentenceTransformer

## Extraire le contenu du csv

In [7]:
import csv
with open("JADe_project1/Data/movie_synopsis.csv") as f:
    content = []
    csv_reader = csv.reader(f)
    for line in csv_reader:
        # process each line
        if line[0] == "":
            print(line)
        else:
            content.append(line)

['', 'synopsis', 'title']


In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

## User Input

In [34]:
input_ju = "mort "
input_jus = model.encode(input_ju)

## Télécharger le modèle et créer les vecteurs

In [10]:
from sentence_transformers import SentenceTransformer
sentences = []
for id,synopsis,title in content:
    # on récupère seulement le synopsis
    sentences.append(synopsis)

# chargement du modèle
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

## Créer un objet contenant les vecteurs et les titres

In [11]:
from collections import namedtuple

# Définir un named tuple pour représenter des coordonnées géographiques
Vecteurs = namedtuple('Vecteurs', ['embedding', 'titre'])

# Créer une instance de named tuple
total = []
for em,(id,synopsis,title) in zip(embeddings,content):
    total.append(Vecteurs(embedding=em, titre=title))

#print(total[1])

## Fonction qui donne les suggestion
Par défaut, on donne 5 suggestions. 

In [11]:
from sklearn.metrics.pairwise import cosine_distances,euclidean_distances
def give_suggestion(user_input:str,k:int=5):
    # on encode l'input de l'user avec le même modèle pour avoir un vecteur
    embed_user = model.encode(user_input)
    stock_similarity = []
    stock_suggest_k = []
    # on calcule les distances cosinus
    for embed,title in total:
        #result = (cosine_distances([embed],[embed_user]),title)
        result = (euclidean_distances([embed],[embed_user]),title)
        # elles sont stockés dans la liste sous forme de tuple contenant la distance cosinus et le titre
        stock_similarity.append(result)
    # on trie (ordre croissant) pour avoir le synopsis le plus proche de l'input de l'usr   
    ma_liste_triee = sorted(stock_similarity, key=lambda x: x[0])
    # afficher les premiers résultats etc - on peut changer cette partie en fonction de ce qu'on veut afficher 
    #(idem pour le return)
    for i, (x, y) in enumerate(ma_liste_triee[:5]):
        print(f"Suggestion {i+1}: {y} - distance {x}")
        stock_suggest_k.append(y)
    return stock_similarity #ou stock_suggest_k ou stock_similarity[:k]

In [12]:
# Test
give_suggestion("C'est l'histoire d'adulte qui parlent et qui s'amusent.")

Suggestion 1: The one I love - distance [[1.0092192]]
Suggestion 2: Magique ! - distance [[1.0526134]]
Suggestion 3: Célibataires - distance [[1.0672598]]
Suggestion 4: Une  approche positive du comportement des perroquets - distance [[1.1119643]]
Suggestion 5: Marie Antoinette - distance [[1.1827376]]


[(array([[1.4576427]], dtype=float32), 'Four Rooms'),
 (array([[1.4429169]], dtype=float32), 'Judgment Night'),
 (array([[1.3898015]], dtype=float32), 'Life in Loops (A Megacities RMX)'),
 (array([[1.3917735]], dtype=float32), 'Star Wars'),
 (array([[1.4357673]], dtype=float32), 'Finding Nemo'),
 (array([[1.4132813]], dtype=float32), 'Forrest Gump'),
 (array([[1.3978893]], dtype=float32), 'American Beauty'),
 (array([[1.3594748]], dtype=float32), 'Citizen Kane'),
 (array([[1.3756926]], dtype=float32), 'Dancer in the Dark'),
 (array([[1.3732328]], dtype=float32), 'The Dark'),
 (array([[1.3686999]], dtype=float32), 'The Fifth Element'),
 (array([[1.3823259]], dtype=float32), 'My Life Without Me'),
 (array([[1.3952595]], dtype=float32), 'The Endless Summer'),
 (array([[1.3745531]], dtype=float32),
  'Pirates of the Caribbean: The Curse of the Black Pearl'),
 (array([[1.3943206]], dtype=float32), 'Kill Bill: Vol. 1'),
 (array([[1.4603466]], dtype=float32), 'Jarhead'),
 (array([[1.35775]], 

# Et avec les KNN ?

In [50]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier



def train_model(donnee:tuple):
    # jeux de données
    X = []
    for emb,_ in total:
        X.append(emb)
    y = [title for _,title in total]
    # créer une instance de LabelEncoder
    label_encoder = LabelEncoder()
    # convertir les labels en entiers
    label_encoder = LabelEncoder()
    label_encoder.fit(y)
    y_encoded = label_encoder.transform(y)

    # entraîner le classificateur KNN avec k=5 (5 voisins)
    # créer un classificateur KNN avec k=5 voisins
    knn = KNeighborsClassifier(n_neighbors=1)

    model = knn.fit(X,y_encoded)
    return model_knn

def give_suggestion_knn(user_input:str, model_knn, k:int=5):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    # on encode l'input de l'user avec le même modèle pour avoir un vecteur
    embed_user = model.encode(user_input)
    predicted_class_encoded = model_knn.predict([embed_user])
    # trouver les 3 voisins les plus proches du point d'entrée
    distances, indices = model_knn.kneighbors([embed_user], n_neighbors=k)
    # extraire les classes des 3 voisins les plus proches
    closest_classes = [y[i] for i in indices[0]]
    return closest_classes

In [51]:
give_suggestion_knn("Deux chats sont amis, ils leur arrivent plein de choses.")

['Magique !',
 'The one I love',
 'Une  approche positive du comportement des perroquets',
 'Uffe Holms Flash back',
 'Célibataires']